### Color Science
- https://www.shadertoy.com/view/4dKcWK
- http://lolengine.net/blog/2013/07/27/rgb-to-hsv-in-glsl
- http://lolengine.net/blog/2013/01/13/fast-rgb-to-hsv
- https://www.chilliant.com/rgb2hsv.html
- HSL(**Hue, Saturation, Lightness**)色相、饱和度、亮度
- HSV(**Hue, Saturation, Value/Brightness**)色相、饱和度、明度/亮度
- Hue 色相,不同波长的光照射下,人眼所感觉到的不同颜色
- Saturation 饱和度 色彩纯度, 饱和度降低色彩变灰

### ToneMap
- https://64.github.io/tonemapping/
- **clamp**:
![](./Images/Tonemap_clamp.png)
- Reinhard:
$$TMO_{reinhard}(C)=\frac{C}{1+C}$$
![](./Images/Tonemap_reinhard.png)
- Extended Reinhard: The problem with the 'simple' Reinhard TMO is that it doesn't necessarily make good use of the full Low Dynamic Range. If our max scene radiance happened to be$(1.0, 1.0, 1.0)$then the resulting maximum brightness would only be$(0.5, 0.5, 0.5)$- only half of the available range. Fortunately, the paper by Reinhard presents a way to scale and make use of the full range: $$TMO_{reinhardext}(C)=\frac{C(1+\frac{C}{C^2_{white}})}{1+C}$$
    - $C_{white}$ is the biggest radiance value in the scene, (Note that you can also just set $C_{white}$ to a value lower than the maximum radiance, which will ensure that anything higher gets mapped to$(1.0, 1.0, 1.0)$ - for this reason it is sometimes referred to as the **white point**.)
- Luminance:
$$L=0.2126R+0.7152G+0.0722B$$
```cpp
half Luminance(half3 linearRgb)
{
    return dot(linearRgb, float3(0.2126729, 0.7151522, 0.0721750));
}
```
- Instead, what Reinhard's formula entails is to convert our linear RGB radiance to luminance, apply tone mapping the luminance, then somehow scale our RGB value by the new luminance. The simplest way of doing that final scaling is:
$$C_{out}=C_{in}\frac{L_{out}}{L_{in}}$$

```cpp
float luminance(vec3 v)
{
    return dot(v, vec3(0.2126f, 0.7152f, 0.0722f));
}

vec3 change_luminance(vec3 c_in, float l_out)
{
    float l_in = luminance(c_in);
    return c_in * (l_out / l_in);
}
```
- Apply the extended Reinhard TMO to luminance only:

```cpp
vec3 reinhard_extended_luminance(vec3 v, float max_white_l)
{
    float l_old = luminance(v);
    float numerator = l_old * (1.0f + (l_old / (max_white_l * max_white_l)));
    float l_new = numerator / (1.0f + l_old);
    return change_luminance(v, l_new);
}
```
![](./Images/Tonemap_reinhard_luminance.png)

- Reinhard-Jodie

https://www.shadertoy.com/user/Jodie

```cpp
vec3 reinhard_jodie(vec3 v)
{
    float l = luminance(v);
    vec3 tv = v / (1.0f + v);
    return lerp(v / (1.0f + l), tv, tv);
}
```
![](./Images/Tonemap_reinhard_jodie.png)

- Filmic Tone Mapping Operators
    - Uncharted 2
    
```cpp
vec3 uncharted2_tonemap_partial(vec3 x)
{
    float A = 0.15f;
    float B = 0.50f;
    float C = 0.10f;
    float D = 0.20f;
    float E = 0.02f;
    float F = 0.30f;
    return ((x*(A*x+C*B)+D*E)/(x*(A*x+B)+D*F))-E/F;
}

vec3 uncharted2_filmic(vec3 v)
{
    float exposure_bias = 2.0f;
    vec3 curr = uncharted2_tonemap_partial(v * exposure_bias);

    vec3 W = vec3(11.2f);
    vec3 white_scale = vec3(1.0f) / uncharted2_tonemap_partial(W);
    return curr * white_scale;
}
```
![](./Images/Tonemap_uncharted2.png)
http://filmicworlds.com/blog/filmic-tonemapping-with-piecewise-power-curves/

- ACES(Academy Color Encoding System)

https://github.com/TheRealMJP/BakingLab/blob/master/BakingLab/ACES.hlsl

```cpp
static const std::array<vec3, 3> aces_input_matrix =
{
    vec3(0.59719f, 0.35458f, 0.04823f),
    vec3(0.07600f, 0.90834f, 0.01566f),
    vec3(0.02840f, 0.13383f, 0.83777f)
};

static const std::array<vec3, 3> aces_output_matrix =
{
    vec3( 1.60475f, -0.53108f, -0.07367f),
    vec3(-0.10208f,  1.10813f, -0.00605f),
    vec3(-0.00327f, -0.07276f,  1.07602f)
};

vec3 mul(const std::array<vec3, 3>& m, const vec3& v)
{
    float x = m[0][0] * v[0] + m[0][1] * v[1] + m[0][2] * v[2];
    float y = m[1][0] * v[1] + m[1][1] * v[1] + m[1][2] * v[2];
    float z = m[2][0] * v[1] + m[2][1] * v[1] + m[2][2] * v[2];
    return vec3(x, y, z);
}

vec3 rtt_and_odt_fit(vec3 v)
{
    vec3 a = v * (v + 0.0245786f) - 0.000090537f;
    vec3 b = v * (0.983729f * v + 0.4329510f) + 0.238081f;
    return a / b;
}

vec3 aces_fitted(vec3 v)
{
    v = mul(aces_input_matrix, v);
    v = rtt_and_odt_fit(v);
    return mul(aces_output_matrix, v);
}
```

![](./Images/Tonemap_aces.png)

- approximated ACES fit by [Krzysztof Narkowicz](https://knarkowicz.wordpress.com/2016/01/06/aces-filmic-tone-mapping-curve/)

```cpp
vec3 aces_approx(vec3 v)
{
    v *= 0.6f;
    float a = 2.51f;
    float b = 0.03f;
    float c = 2.43f;
    float d = 0.59f;
    float e = 0.14f;
    return clamp((v*(a*v+b))/(v*(c*v+d)+e), 0.0f, 1.0f);
}
```